In [1]:
import random
import torch
import numpy as np


def setup_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")
    
SEED = 1000
setup_reproducibility(SEED)

In [2]:
from transformers import get_cosine_schedule_with_warmup
from scipy import signal
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from huggingface_hub import login, snapshot_download
from tqdm.auto import tqdm


def get_scheduler(optimizer, train_dl, epochs):
    total_training_steps = len(train_dl) * epochs
    warmup_steps = int(total_training_steps * 0.05)  # e.g. 5% warmup
    
    return get_cosine_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_training_steps
    )


def get_stats(tensor, p=True, r=False):
    mean, std = tensor.mean(), tensor.std()
    min, max =  tensor.min(), tensor.max()
    
    #if p: print(f"Min: {min}, Max: {max}, Mean: {mean}, Std: {std}")
    if p: print(f"Mean: {mean}, Std: {std}")
    if r: return mean, std
    
    
def zscore(tensor, mean=None, std=None):
    if mean is None: mean = tensor.mean()
    if std is None: std = tensor.std()
    return (tensor - mean) / (std + 1e-8)


def get_model_size(model):
    print(sum(p.numel() for p in model.parameters()) / 1e6)
    

def get_index(iterable):
    return random.randint(0, len(iterable) - 1)


def get_indices(iterable, n):
    return random.sample(range(len(iterable)), n)


def split(inputs, targets, seed):
    return train_test_split(
        inputs,
        targets, 
        test_size=0.2,
        shuffle=True, 
        random_state=seed
    ) 


def show_waves(waves, dpi=100):
    """
    waves: numpy array of shape (3, N)
    Creates three separate figures that stretch wide.
    """
    N = waves.shape[1]
    t = np.arange(N)

    # Wide aspect ratio; height modest so each window fills width
    for i in range(waves.shape[0]):
        fig = plt.figure(figsize=(14, 4), dpi=dpi)  # wide figure
        ax = fig.add_subplot(111)
        ax.plot(t, waves[i], linewidth=1)
        ax.set_title(f"Wave {i+1}")
        ax.set_xlabel("Sample")
        ax.set_ylabel("Amplitude")
        ax.grid(True)
        fig.tight_layout()  # reduce margins to use width
        
    plt.show()
    
    
def hf_ds_download(hf_token, repo_id):
    login(hf_token[1:])
    return snapshot_download(repo_id, repo_type="dataset")


def get_spectra_features(X, b=False):
    """Create multi-channel features from spectra: raw, 1st derivative, 2nd derivative."""
    X_processed = np.zeros_like(X)
    # Baseline correction and SNV
    for i in tqdm(range(X.shape[0])):
        poly = np.polyfit(np.arange(X.shape[1]), X[i], 3)
        baseline = np.polyval(poly, np.arange(X.shape[1]))
        corrected_spec = X[i] - baseline
        #X_processed[i] = (corrected_spec - corrected_spec.mean()) / (corrected_spec.std() + 1e-8)
        X_processed[i] = corrected_spec
        
    # Calculate derivatives
    deriv1 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=1, axis=1)
    deriv2 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=2, axis=1)

    if b: return np.stack([X_processed, deriv1, deriv2], axis=1)
    return np.stack([deriv1, deriv2], axis=1)

In [3]:
import os

path = "/kaggle/input/dig-4-bio-raman-transfer-learning-challenge"
files = os.listdir(path)
[(i, files[i]) for i in range(len(files))]

[(0, 'sample_submission.csv'),
 (1, 'timegate.csv'),
 (2, 'mettler_toledo.csv'),
 (3, 'kaiser.csv'),
 (4, 'anton_532.csv'),
 (5, 'transfer_plate.csv'),
 (6, '96_samples.csv'),
 (7, 'tornado.csv'),
 (8, 'tec5.csv'),
 (9, 'metrohm.csv'),
 (10, 'anton_785.csv')]

In [4]:
import pandas as pd


def load_transfer_data():
    csv_path = os.path.join(path, files[5])
    df = pd.read_csv(csv_path)

    input_cols = df.columns[1:2049]
    target_cols = df.columns[2050:]

    targets  = df[target_cols].dropna().to_numpy()

    df = df[input_cols]
    df['Unnamed: 1'] = df['Unnamed: 1'].str.replace("[\[\]]", "", regex=True).astype('int64')
    df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace("[\[\]]", "", regex=True).astype('int64')

    inputs = df.to_numpy().reshape(-1, 2, 2048)
    inputs = inputs.mean(axis=1)

    return inputs, targets


def load_test_data(path):
    test = pd.read_csv(os.path.join(path, files[6]))

    row1 = test.columns[1:].to_numpy().copy()
    row1[-1] = "5611"
    row1 = row1.astype(np.float64)


    cols = test.columns[1:]
    test = test[cols]
    test[" 5611]"] = test[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')
    test = test.to_numpy()

    test = np.insert(test, 0, row1, axis=0)
    return test.reshape(-1, 2, 2048).mean(axis=1)

In [5]:
inputs, targets = load_transfer_data()
inputs = get_spectra_features(inputs)
inputs = torch.tensor(inputs)
targets = torch.tensor(targets)
#train_inputs, eval_inputs, train_targets, eval_targets = split(inputs, targets, SEED)

  0%|          | 0/96 [00:00<?, ?it/s]

In [6]:
if False:
    train_inputs = torch.tensor(train_inputs)
    eval_inputs = torch.tensor(eval_inputs)
    train_targets = torch.tensor(train_targets)
    eval_targets = torch.tensor(eval_targets)

In [7]:
if False:
    min, max, mu, sigma = get_stats(train_inputs, r=True)
    train_inputs = zscore(train_inputs)
    eval_inputs = zscore(eval_inputs)
    get_stats(train_inputs), get_stats(eval_inputs)

In [8]:
from torch.utils.data import TensorDataset

if False:
    train_ds = TensorDataset(train_inputs.float(), train_targets.float())
    eval_ds = TensorDataset(eval_inputs.float(), eval_targets.float())
    len(train_ds), len(eval_ds)

In [9]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+5232)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
def return_dls(train_ds, eval_ds, train_batch_size, eval_batch_size):
    train_dl = build_loader(
        SEED,
        train_ds,
        train=True,
        batch_size=train_batch_size,
        shuffle=True,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )

    eval_dl = build_loader(
        SEED,
        eval_ds,
        train=False,
        batch_size=eval_batch_size,
        shuffle=False,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )
    
    return train_dl, eval_dl

In [10]:
import neptune


def setup_neptune():
    if not RESUME:
        neptune_run = neptune.init_run(
            project="arbaaz/kaggle-spect",
            name=MODEL_NAME,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

        neptune_run["h_parameters"] = {
            "seed": SEED,
            "model_name": MODEL_NAME,
            "optimizer_name": "nadam",
            "learning_rate": LR,
            "scheduler_name": "default",
            "weight_decay": WD,
            "num_epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
        }
        if DROPOUT: neptune_run["h_parameters"] = {"dropout": DROPOUT}
        if DROP_PATH_RATE: neptune_run["h_parameters"] = {"drop_path_rate": DROP_PATH_RATE}
    else:
        neptune_run = neptune.init_run(
            project="arbaaz/crunchdao-structural-break",
            with_id=config.with_id,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

    return neptune_run

In [11]:
import torch.nn.functional as F
from sklearn.metrics import r2_score


def loss_fn(logits, targets):
    logits = logits.view(-1)
    targets = targets.view(-1)
    return F.mse_loss(logits, targets)


def metric_fn(logits, targets):
    preds = logits.cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    
    dim1 = r2_score(targets[:, 0], preds[:, 0])
    dim2 = r2_score(targets[:, 1], preds[:, 1])
    dim3 = r2_score(targets[:, 2], preds[:, 2])
    
    mean_r2 = (dim1 + dim2 + dim3) / 3
    
    return dim1, dim2, dim3, mean_r2

In [12]:
import torch.nn as nn


class ResidualBlock(nn.Module):
    """A residual block with two 1D convolutional layers."""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding=kernel_size//2)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.elu = nn.ELU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=kernel_size//2)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        out = self.elu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.elu(out)
        return out

class ResNet(nn.Module):
    """A deeper ResNet-style 1D CNN for Raman spectra."""
    def __init__(self, dropout, input_channels=3, num_classes=3):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.elu = nn.GELU()
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ELU(),
            nn.Dropout(dropout), # Increased dropout for better regularization
            nn.Linear(256, num_classes)
        )

    def _make_layer(self, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for s in strides:
            layers.append(ResidualBlock(self.in_channels, out_channels, stride=s))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.elu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [13]:
from tqdm.auto import tqdm


def train(
    model, 
    optimizer,
    device,
    scaler, 
    scheduler,
    train_dl,
    eval_dl,
    epochs,
    checkpoint_name,
    score=-float("inf"),
    neptune_run=None,
    p=True,
):  
    for epoch in tqdm(range(epochs)):
        model.train()
        total_loss = 0.0
        all_logits = []
        all_targets = []
        
        for inputs, targets in train_dl:
            inputs = inputs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            
            with torch.amp.autocast(device_type=device, dtype=torch.float16, cache_enabled=True):
                logits = model(inputs)
                loss = loss_fn(logits, targets)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
            if neptune_run is not None:  neptune_run["lr_step"].append(scheduler.get_last_lr()[0])
            
            total_loss += loss.detach().cpu()
            all_logits.append(logits.detach().cpu())
            all_targets.append(targets.detach().cpu())
        
        all_logits = torch.cat(all_logits)
        all_targets = torch.cat(all_targets)

        one, two, three, r2 = metric_fn(all_logits, all_targets)
        total_loss = total_loss / len(train_dl)
        
        model.eval()
        eval_total_loss = 0.0
        eval_all_logits = []
        eval_all_targets = []

        for inputs, targets in eval_dl:
            inputs = inputs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)

            with torch.inference_mode():
                #with torch.amp.autocast(device_type=device, dtype=torch.float16, cache_enabled=True):
                logits = model(inputs)
                loss = loss_fn(logits, targets)

            eval_total_loss += loss.detach().cpu()
            eval_all_logits.append(logits.detach().cpu())
            eval_all_targets.append(targets.detach().cpu())
        
        eval_all_logits = torch.cat(eval_all_logits)
        eval_all_targets = torch.cat(eval_all_targets)

        eval_one, eval_two, eval_three, eval_r2 = metric_fn(eval_all_logits, eval_all_targets)
        eval_total_loss = eval_total_loss / len(eval_dl)
        
        if eval_r2 > score:
            score = eval_r2
            data = {"state_dict": model.state_dict()}
            data["epoch"] = epoch 
            data["score"] = score
            torch.save(data, f"/kaggle/working/{checkpoint_name}")
        
        if neptune_run is not None:
            neptune_run["train/loss"].append(total_loss)
            neptune_run["eval/loss"].append(eval_total_loss)
            neptune_run["train/r2"].append(r2)
            neptune_run["eval/r2"].append(eval_r2)
            neptune_run["train/one"].append(one)
            neptune_run["train/two"].append(two)
            neptune_run["train/three"].append(three)
            neptune_run["eval/one"].append(eval_one)
            neptune_run["eval/two"].append(eval_two)
            neptune_run["eval/three"].append(eval_three)
            
        if p and epoch % 5 == 0:
            print(
                f"Epoch: {epoch}, "
                f"train/loss: {total_loss:.4f}, "
                f"eval/loss: {eval_total_loss:.4f}, "
                f"train/r2: {r2:.4f}, "
                f"eval/r2: {eval_r2:.4f}, "
                f"train/one: {one:.4f}, "
                f"train/two: {two:.4f}, "
                f"train/three: {three:.4f}, "
                f"eval/one: {eval_one:.4f}, "
                f"eval/two: {eval_two:.4f}, "
                f"eval/three: {eval_three:.4f} "
            )
            
    if neptune_run is not None: neptune_run.stop()
    return score

In [14]:
import warnings#; warnings.filterwarnings("ignore")


EPOCHS = 500
WD = 1e-3
LR = 1e-4

DROPOUT = 0.5
DROP_PATH_RATE = None

device = "cuda" if torch.cuda.is_available() else "cpu"
RESUME = False

In [16]:
from sklearn.model_selection import KFold


mus = [0.5431315108096543,
 0.5373212199292232,
 0.5625938858971476,
 0.5611233230776563,
 0.5471325694126035]

sigmas = [249.05215723502948,
 248.6726517269414,
 247.84789229685813,
 247.65868506084584,
 248.16012425733976]

mus, sigmas = [], []
scores = []
kfold = KFold(n_splits=5, shuffle=True, random_state=SEED)
splits = kfold.split(range(96))

for fold, (train_idx, eval_idx) in enumerate(splits):
    MODEL_NAME = f"resnet.finetune.fold.{fold}"
    checkpoint_name = f"finetune.fold.{fold}.pt"
    
    train_inputs = inputs[train_idx]
    train_targets = targets[train_idx]
    eval_inputs = inputs[eval_idx]
    eval_targets = targets[eval_idx]
    
    mu, sigma = get_stats(train_inputs, p=False, r=True)
    train_inputs = zscore(train_inputs, mu, sigma)
    eval_inputs = zscore(eval_inputs, mu, sigma)
    mus.append(mu)
    sigmas.append(sigma)
    continue
    train_ds = TensorDataset(train_inputs.float(), train_targets.float())
    eval_ds = TensorDataset(eval_inputs.float(), eval_targets.float())
    
    BATCH_SIZE = len(train_ds)
    train_dl, eval_dl = return_dls(train_ds, eval_ds, BATCH_SIZE, len(eval_ds))
    
    model = ResNet(input_channels=2, dropout=DROPOUT).to(device)
    if fold == 0: print(get_model_size(model))
    
    ckpt = torch.load(f"pretrain.fold.{fold}.pt", weights_only=False)
    print(ckpt["score"])
    model.load_state_dict(ckpt["state_dict"])
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD, foreach=True)
    scaler = torch.amp.GradScaler(device)
    scheduler = get_scheduler(optimizer, train_dl, EPOCHS)
    score = train(
            model, 
            optimizer, 
            device,
            scaler,
            scheduler,
            train_dl, 
            eval_dl,
            EPOCHS,
            checkpoint_name,
            neptune_run=setup_neptune(),
        )
    
    scores.append(score)

In [18]:
files_

['.neptune',
 '.virtual_documents',
 'finetune.fold.0.pt',
 'finetune.fold.1.pt',
 'finetune.fold.2.pt',
 'finetune.fold.3.pt',
 'finetune.fold.4.pt',
 'pretrain.fold.0.pt',
 'pretrain.fold.1.pt',
 'pretrain.fold.2.pt',
 'pretrain.fold.3.pt',
 'pretrain.fold.4.pt',
 'resnet.finetune.8823.fold1.csv',
 'resnet.pretrain.4882.fold1.csv']

In [29]:
import os
import torch

core_path = "/kaggle/working"
files_ = sorted(os.listdir(core_path))
i = 0
all_preds = []
for f in files_:
    if "finetune.fold" in f:
        ckpt_path = os.path.join(core_path, f)
        ckpt = torch.load(ckpt_path, weights_only=False)

        test = load_test_data(path)
        test = get_spectra_features(test)
        test = torch.tensor(test)
        test = zscore(test, mus[i], sigmas[i]).float()
        i += 1

        model = ResNet(input_channels=2, dropout=0.5).to(device)
        model.load_state_dict(ckpt["state_dict"])
        model.eval()

        with torch.inference_mode():
            preds = model(test.cuda())

        preds = preds.cpu().detach().double()
        all_preds.append(preds)

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

In [30]:
preds = torch.stack(all_preds)

In [31]:
preds = preds.mean(0).numpy()

In [34]:
ckpt = torch.load("/kaggle/working/pretrain.fold.1.pt", weights_only=False)
ckpt["score"]

0.48822867684523436

In [35]:
test = load_test_data()
get_stats(test)
test = get_spectra_features(test)
test = torch.tensor(test)
test = zscore(test, 0.5373212199292232, 248.6726517269414).float()
test.shape, test.dtype, get_stats(test)

Mean: 3623.216623942057, Std: 6772.114021862655


  0%|          | 0/96 [00:00<?, ?it/s]

Mean: 0.02771512232720852, Std: 2.6981143951416016


(torch.Size([96, 2, 2048]), torch.float32, None)

In [36]:
model = ResNet(input_channels=2, dropout=0.5).to(device)
model.load_state_dict(ckpt["state_dict"])
model.eval()

with torch.inference_mode():
    preds = model(test.cuda())

preds = preds.cpu().detach().double().numpy()
preds.shape, get_stats(preds)

Mean: 2.440112045396947, Std: 2.335613724637091


((96, 3), None)

In [32]:
preds.min(), preds.max(), get_stats(preds)

Mean: 2.2413799092406403, Std: 1.9773549545086004


(0.3601879395544529, 9.246828269958495, None)

In [33]:
column_names = ['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']
preds_df = pd.DataFrame(preds, columns=column_names)
preds_df.insert(0, 'ID', [i+1 for i in range(len(preds_df))])
preds_df

,ID,Glucose,Sodium Acetate,Magnesium Sulfate
0,1,2.679680,0.783572,0.458006
1,2,6.443715,1.512056,1.462726
2,3,4.334267,0.498753,1.138379
3,4,3.869976,0.856771,0.403803
4,5,9.246828,0.846140,1.014109
...,...,...,...,...
91,92,3.329243,0.795816,1.240150
92,93,3.728871,0.473928,0.961597
93,94,4.097541,0.600958,0.902222
94,95,3.378495,1.356923,0.916949


In [39]:
MODEL_NAME, ckpt["score"]

('resnet.finetune.fold.4', 0.48822867684523436)

In [34]:
name = "resnet.finetune.all.folds.csv"
preds_df.to_csv(name, index=False)
f = pd.read_csv(f"/kaggle/working/{name}")
f

,ID,Glucose,Sodium Acetate,Magnesium Sulfate
0,1,2.679680,0.783572,0.458006
1,2,6.443715,1.512056,1.462726
2,3,4.334267,0.498753,1.138379
3,4,3.869976,0.856771,0.403803
4,5,9.246828,0.846140,1.014109
...,...,...,...,...
91,92,3.329243,0.795816,1.240150
92,93,3.728871,0.473928,0.961597
93,94,4.097541,0.600958,0.902222
94,95,3.378495,1.356923,0.916949


In [35]:
preds

array([[2.67967954, 0.78357179, 0.45800613],
       [6.44371519, 1.51205604, 1.46272559],
       [4.33426738, 0.4987534 , 1.13837928],
       [3.86997619, 0.856771  , 0.40380331],
       [9.24682827, 0.84614031, 1.01410854],
       [7.95932093, 1.74550412, 1.12293968],
       [4.92644253, 0.6653578 , 0.36018794],
       [8.07706261, 1.68925283, 1.35848229],
       [6.14031515, 1.24895676, 1.24107531],
       [8.34203043, 1.0242914 , 0.49165687],
       [7.09966297, 1.09119189, 1.17774593],
       [3.86741662, 0.67115462, 1.05267402],
       [3.2626493 , 0.88938384, 1.09069486],
       [3.85280752, 0.70643659, 1.44978414],
       [3.14084277, 0.75801558, 1.07351474],
       [6.3768589 , 0.97297016, 0.99683253],
       [3.77613826, 0.76518672, 1.10728935],
       [5.3258625 , 0.97261331, 1.16698685],
       [5.0781498 , 1.08894435, 1.17013738],
       [2.64447912, 0.72874848, 1.14038044],
       [3.38296123, 0.85824862, 1.00850284],
       [4.64680042, 0.84010615, 1.06851076],
       [2.